## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.models.cifar10.wideresnet import wrn_22
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512
sz=32
workers=7

In [4]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def pad(img, p=4, padding_mode='reflect'):
        return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))
def to_pil(img): return Image.fromarray(img)

def torch_tfms(size, conv_pil=False, to_numpy=False):
    mean,std=[0.4914 , 0.48216, 0.44653], [0.24703, 0.24349, 0.26159]
    normalize = transforms.Normalize(mean=mean, std=std)
    tfms = [transforms.ToTensor(), normalize]
    
    # Torch transforms with fastai dl
    if to_numpy: tfms = [np.array, Normalize(mean,std), lambda x: x[0].T]
        
    aug_tfms = [
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
    ]
    scale_size = 40
    padding = int((scale_size - size) / 2)
    
    train_tfms = transforms.Compose(aug_tfms + tfms)
    train_tfms.sz = size
    val_tfms = transforms.Compose(tfms)
    val_tfms.sz = size
    if conv_pil:
        train_tfms.transforms.insert(0, to_pil)
        val_tfms.transforms.insert(0, to_pil)
    return train_tfms, val_tfms
    
def torch_ds(data_path, tfms):
    train_tfms, val_tfms = tfms
    
    # Data loading code
    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'test')

    train_dataset = datasets.ImageFolder(traindir, train_tfms)
    val_dataset = datasets.ImageFolder(valdir, val_tfms)
    return train_dataset, val_dataset

def torch_loader(data_path, datasets):
    train_dataset, val_dataset = datasets
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs*2, shuffle=False,
        num_workers=workers, pin_memory=True)

    data = ModelData(data_path, train_loader, val_loader)
    return data


In [5]:
def get_learner(data):
    m = wrn_22()
    learn = ConvLearner.from_model_data(m, data)
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    return learn

### Torch Data Loader

In [6]:
tfms = torch_tfms(sz)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [7]:
learn = get_learner(torch_data)

In [8]:
# Warmup session
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                 
    0      2.220326   2.10418    0.1978    

CPU times: user 17.8 s, sys: 9.77 s, total: 27.6 s
Wall time: 31.4 s


[array([2.10418]), 0.1978]

In [9]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.009644   1.923166   0.2344    
    1      1.914014   1.838584   0.2715                   

CPU times: user 34.6 s, sys: 18.7 s, total: 53.4 s
Wall time: 54.9 s


[array([1.83858]), 0.2714999995946884]

### Fastai Data Loader

In [11]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [12]:
learn = get_learner(fastai_data)

In [13]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.31463    2.272603   0.1246    
    1      2.161199   2.113671   0.1875                   

CPU times: user 2min 15s, sys: 3min 42s, total: 5min 57s
Wall time: 1min 6s


[array([2.11367]), 0.187500000166893]

### Fastai Default Collate

In [14]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_dc_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [15]:
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = torch.utils.data.DataLoader(None).collate_fn

In [16]:
fastai_dc_data.trn_dl.collate_fn = collate_fn
fastai_dc_data.val_dl.collate_fn = collate_fn

In [17]:
learn = get_learner(fastai_dc_data)

In [18]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.105816   1.978059   0.2379    
    1      1.968836   1.918824   0.2612                   

CPU times: user 2min 16s, sys: 3min 38s, total: 5min 54s
Wall time: 1min 5s


[array([1.91882]), 0.26119999985694886]

### Fastai DataLoader - Torch Transforms

In [20]:
tfms = torch_tfms(sz, to_numpy=True)

In [21]:
fdl_tds_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [22]:
fdl_tds_data.trn_ds.open_fn = Image.open
fdl_tds_data.val_ds.open_fn = Image.open

In [23]:
learn = get_learner(fdl_tds_data)

In [24]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.085024   2.094643   0.2199    
    1      1.878126   1.761554   0.3321                   

CPU times: user 3min 26s, sys: 4min 13s, total: 7min 39s
Wall time: 2min 32s


[array([1.76155]), 0.33209999980926513]

### Torch DataLoader - Fastai Datasets

In [25]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)
tdl_fds = fdl.trn_ds, fdl.val_ds
torch_data = torch_loader(str(PATH), tdl_fds)

In [30]:
learn = get_learner(torch_data)

In [31]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.23888    2.134263   0.1867    
    1      1.998574   1.913979   0.2573                   

CPU times: user 34 s, sys: 19.6 s, total: 53.7 s
Wall time: 54.1 s


[array([1.91398]), 0.25729999997615816]

### Fastai DataLoader - Torch Dataset - Same as Torch Transforms

In [ ]:
# from fastai.dataset import FilesArrayDataset, folder_source, read_dir

In [45]:
tfms = torch_tfms(sz, to_numpy=True)
dsets = torch_ds(PATH, tfms)

In [46]:
fdl_tds_data = ImageData(PATH, list(dsets)+[None,None,None,None], bs, num_workers=workers, classes=dsets[0].classes)

In [47]:
learn = get_learner(fdl_tds_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)